In [4]:
# import torch
# from transformers import BertForSequenceClassification, BertTokenizer

# # Load tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# num_labels = 4


# # Load the model architecture
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# # Load the saved model state dictionary
# model_path = "bert_sentiment_model.pth"
# model.load_state_dict(torch.load(model_path))

# # Set the model to evaluation mode
# model.eval()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [8]:
!pip install streamlit

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.


     ---------------------------------------- 8.5/8.5 MB 783.4 kB/s eta 0:00:00
     ---------------------------------------- 6.9/6.9 MB 1.5 MB/s eta 0:00:00
     ------------------------------------ 207.3/207.3 kB 842.3 kB/s eta 0:00:00
     ------------------------------------ 413.4/413.4 kB 920.4 kB/s eta 0:00:00
     -------------------------------------- 240.7/240.7 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 857.8/857.8 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 25.9/25.9 MB 677.2 kB/s eta 0:00:00
     ---------------------------------------- 62.7/62.7 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 913.0 kB/s eta 0:00:00
     -------------------------------------- 87.5/87.5 kB 978.3 kB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1
  Attempting uninstall: pygments
    Found

In [16]:
pip install wikipedia-api


In [22]:
%%writefile interface.py
import streamlit as st
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from fuzzywuzzy import fuzz
import requests
from bs4 import BeautifulSoup  # Import BeautifulSoup for HTML parsing

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
num_labels = 4

# Load the model architecture
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Load the saved model state dictionary
model_path = "bert_sentiment_model.pth"
model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
model.eval()

# Define labels and label encoder
labels = ["fraud", "drug", "kidnap", "murder"]
label_encoder = LabelEncoder()
label_encoder.fit(labels)

# Load your judge data
judge = pd.read_csv("LatJudge.csv")

# Define a function to predict labels
def predict_label(model, tokenizer, text):
    # Tokenize the input text
    encoded_text = tokenizer.encode_plus(
        text,
        max_length=128,  # Adjust as needed based on your model's input size
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    # Move input tensors to the appropriate device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = encoded_text['input_ids'].to(device)
    attention_mask = encoded_text['attention_mask'].to(device)

    # Make predictions
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Convert logits to predicted labels
    _, predicted_label = torch.max(logits, 1)

    # Decode predicted label using label encoder
    predicted_label = predicted_label.item()
    predicted_label_text = label_encoder.inverse_transform([predicted_label])[0]

    return predicted_label_text

# Function to filter judge data and calculate similarity scores
def process_data_and_similarity(input_text, predicted_label_text, judge_data):
    # Filter the judge data based on predicted label
    filtered_df = judge_data[judge_data['Name'].str.lower().str.contains(predicted_label_text.lower())]

    # Calculate similarity scores using fuzzywuzzy
    def get_similarity_score(row):
        return fuzz.token_sort_ratio(input_text.lower(), row['Name'].lower())

    # Apply the function to the DataFrame to get similarity scores
    filtered_df['Similarity'] = filtered_df.apply(get_similarity_score, axis=1)

    # Get top 5 most similar instances
    top_similar = filtered_df.nlargest(5, 'Similarity')

    return filtered_df, top_similar

# Function to fetch information from Wikipedia and clean HTML tags
def fetch_wikipedia_info(question):
    # Define custom headers with user agent
    headers = {
        'User-Agent': 'Your-User-Agent-Name/1.0'  # Replace 'Your-User-Agent-Name' with your actual user agent
    }

    # Make a request to the Wikipedia API with custom headers
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&exintro=&titles={question}"
    response = requests.get(url, headers=headers)

    # Check if the request was successful and process the response
    if response.status_code == 200:
        data = response.json()
        pages = data['query']['pages']
        page_id = list(pages.keys())[0]
        page_data = pages[page_id]
        if 'extract' in page_data:
            # Parse HTML content using BeautifulSoup to remove tags
            soup = BeautifulSoup(page_data['extract'], 'html.parser')
            return soup.get_text()
        else:
            return "No information found on Wikipedia."
    else:
        return "Error fetching data from Wikipedia."

# Streamlit UI
st.title("Crime Category Prediction")

# Input text box for user input
user_input = st.text_input("Enter a sentence describing a crime:")

# Button to trigger prediction and processing
if st.button("Predict and Process"):
    if user_input.strip() == "":
        st.error("Please enter a sentence.")
    else:
        # Predict the label
        predicted_label = predict_label(model, tokenizer, user_input)
        st.success(f"Predicted Label: {predicted_label}")

        # Process data and calculate similarity
        filtered_data, top_similar_instances = process_data_and_similarity(user_input, predicted_label, judge)

        # Display the filtered data and top similar instances
        st.write("Filtered Data:")
        st.write(filtered_data)

        st.write("Top 5 most similar instances:")
        st.write(top_similar_instances)

# Input box for Wikipedia question
wiki_question = st.text_input("Ask a question to fetch information from Wikipedia:")

# Button to trigger Wikipedia information fetching
if st.button("Fetch Wikipedia Info"):
    if wiki_question.strip() == "":
        st.error("Please enter a question.")
    else:
        # Fetch information from Wikipedia
        wiki_info = fetch_wikipedia_info(wiki_question)
        st.write("Wikipedia Information:")
        st.write(wiki_info)


Writing interface.py
